# Effective Prompting Techniques for Extended Thinking

This notebook explores advanced prompting techniques to maximize the effectiveness of Claude 3.7 Sonnet's extended thinking capability. Building on what we've learned in previous lessons about when and how to use extended thinking, we'll now focus on *how* to craft prompts that get the best results.

In this lesson, you'll learn:

1. The differences between general and step-by-step instruction patterns
2. How to implement N-shot prompting with extended thinking
3. Techniques to control reasoning depth and focus
4. How to create reusable prompt templates for different use cases

By the end of this notebook, you'll have practical strategies for crafting prompts that leverage Claude's extended thinking capabilities to their fullest potential.

## Setup

First, let's import our utility functions and set up the Bedrock clients for working with Claude 3.7 Sonnet:

In [ ]:
# Import required libraries
import boto3
import json
import time
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, HTML

# Import our utility functions from previous lessons
import claude_utils

# Set up the Bedrock clients using our utility module
REGION = 'us-west-2'  # Change to your preferred region
bedrock, bedrock_runtime = claude_utils.create_bedrock_clients(REGION)

# Claude 3.7 Sonnet model ID
CLAUDE_37_SONNET_MODEL_ID = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'

# Verify model availability
claude_utils.verify_model_availability(bedrock, CLAUDE_37_SONNET_MODEL_ID)

## 1. General vs. Step-by-Step Instruction Patterns

When working with Claude's extended thinking capability, there are two main approaches to providing instructions:

1. **General Instructions**: Broad, open-ended guidance that gives Claude freedom to determine its own thinking approach
2. **Step-by-Step Instructions**: Prescriptive, sequential guidance that outlines a specific thinking process

Each approach has its strengths and ideal use cases. Let's explore both strategies and see how they affect Claude's reasoning process.

### General Instructions Approach

When using Claude's extended thinking capability, **general instructions often yield better results** than prescriptive step-by-step guidance. This might seem counterintuitive at first, but it makes sense when you consider how extended thinking works:

![General vs. Step-by-Step instructions](./images/lesson4/general_vs_sbs.png)

With extended thinking enabled, Claude has the space to work through problems in a thorough, methodical way. By giving general instructions to "think deeply" rather than prescribing exact steps, you allow Claude to leverage its own problem-solving capabilities, which may exceed what a human might think to prescribe.

Let's see this approach in action with a challenging problem:

In [ ]:
# Define a problem to solve
problem = """
A retailer sells two products, Product A and Product B. 
Product A costs $20 to make and sells for $50. 
Product B costs $10 to make and sells for $25. 
The retailer has $2000 available for production costs, 
and warehouse space for at most 250 units in total.

Due to customer demand, they need to produce at least twice as many units of Product B as Product A.

What is the optimal production plan to maximize profit?
"""

# Define an effective general instruction that leverages extended thinking
general_instruction = """
Please think about this optimization problem thoroughly and in great detail.
Consider multiple approaches and show your complete reasoning.
"""

# Create the full prompt
full_prompt = f"{general_instruction}\n\n{problem}"

# Invoke Claude with extended thinking
response = claude_utils.invoke_claude(
    bedrock_runtime,
    full_prompt,
    CLAUDE_37_SONNET_MODEL_ID,
    enable_reasoning=True,
    reasoning_budget=4096,
    max_tokens=1200
)

# Display the response
claude_utils.display_claude_response(response)

### When General Instructions Work Best

The example above demonstrates how Claude can effectively approach complex problems with general instructions when extended thinking is enabled. This approach works particularly well for:

- **Open-ended problems** where multiple solution paths exist
- **Complex scenarios** where the optimal approach isn't immediately obvious
- **Creative problem-solving** where innovative thinking is valuable
- **Multi-constraint problems** that require balancing different factors

When using extended thinking with Claude 3.7, you'll generally get better results by:

1. Giving broad guidance about the thoroughness and depth expected
2. Encouraging consideration of multiple approaches
3. Allowing Claude to structure its own thinking process
4. Focusing on what to accomplish rather than exactly how to do it

That said, Claude can still effectively follow structured execution steps when needed. The model can handle even longer lists with more complex instructions than previous versions.

In [ ]:
# Let's create a helper function to demonstrate different instruction styles
def test_instruction_style(problem, instruction_style, reasoning_budget=4096):
    """
    Test a specific instruction style with Claude's extended thinking
    
    Args:
        problem (str): The problem to solve
        instruction_style (str): The instruction to prefix the prompt
        reasoning_budget (int): The reasoning budget to allocate
        
    Returns:
        dict: The full API response
    """
    full_prompt = f"{instruction_style}\n\n{problem}"
    
    print(f"Testing with instruction style: \n{instruction_style}\n")
    response = claude_utils.invoke_claude(
        bedrock_runtime,
        full_prompt,
        CLAUDE_37_SONNET_MODEL_ID,
        enable_reasoning=True,
        reasoning_budget=reasoning_budget,
        max_tokens=1000
    )
    
    # Get just the first 300 characters of the response for a preview
    result_text = claude_utils.extract_response_content(response)
    preview = result_text[:300] + "..." if len(result_text) > 300 else result_text
    
    print("\nResponse preview:")
    print("-" * 80)
    print(preview)
    print("-" * 80)
    
    return response

In [ ]:
# Now let's try different instruction styles on a new problem

creative_problem = """
A city is experiencing increasing traffic congestion in its downtown area. 
The mayor has asked for innovative solutions that would reduce traffic by at least 20% 
while staying within a budget of $5 million and being implementable within 18 months.
What approach would you recommend?
"""

# First, test with a general, open-ended instruction
general_creative_instruction = """
Please think deeply and thoroughly about this urban planning challenge.
Consider multiple approaches, evaluate their pros and cons, and develop
a comprehensive recommendation.
"""

general_response = test_instruction_style(creative_problem, general_creative_instruction)

In [ ]:
# For comparison, let's try a more prescriptive approach
prescriptive_instruction = """
Analyze this urban planning problem using the following steps:
1. Identify at least 3 main causes of congestion
2. List exactly 5 potential solutions that fit the budget
3. Calculate the estimated traffic reduction for each solution
4. Create a table comparing all solutions by cost, timeline, and impact
5. Select the top solution and justify it
6. Outline an implementation plan with specific milestones
"""

prescriptive_response = test_instruction_style(creative_problem, prescriptive_instruction)

# Let's compare the token usage and timing for both approaches
general_tokens = general_response.get('usage', {}).get('totalTokens', 0)
general_time = general_response.get('_elapsed_time', 0)

prescriptive_tokens = prescriptive_response.get('usage', {}).get('totalTokens', 0)
prescriptive_time = prescriptive_response.get('_elapsed_time', 0)

print("\nComparison of approaches:")
print(f"General approach: {general_tokens} tokens in {general_time:.2f} seconds")
print(f"Prescriptive approach: {prescriptive_tokens} tokens in {prescriptive_time:.2f} seconds")

### Key Insight: Balance is Important

While general instructions typically work better with extended thinking, there are scenarios where some structure is helpful. The key is finding the right balance:

- **Too prescriptive**: May constrain Claude's thinking and miss creative solutions
- **Too vague**: May not provide enough guidance for focused problem-solving

**Best Practice**: When using extended thinking, start with general instructions that encourage thorough reasoning. Only add structural guidance when it directly benefits the specific task at hand.

Let's look at a more balanced approach that provides some structure without being overly prescriptive:

In [ ]:
# A balanced approach that provides some structure without being overly prescriptive
balanced_instruction = """
Please think deeply about this urban planning challenge and develop a comprehensive recommendation.

In your reasoning, consider:
- Various causes of downtown congestion
- A diverse range of potential solutions
- How each solution impacts different stakeholders
- Implementation feasibility within the constraints

Develop your thinking thoroughly before reaching a conclusion.
"""

balanced_response = test_instruction_style(creative_problem, balanced_instruction)

# Display full response for the balanced approach
print("\n\nFull response from balanced approach:")
claude_utils.display_claude_response(balanced_response)

## Check-in Point: General Instructions and Extended Thinking

Before we continue, let's check our understanding:

1. When using Claude's extended thinking capability, **general instructions** typically yield better results than highly prescriptive step-by-step guidance
2. With extended thinking, Claude has the space to work through problems methodically, so we can focus on **what** to accomplish rather than exactly **how** to do it
3. A balanced approach provides some helpful context without constraining Claude's problem-solving capabilities
4. Claude can still follow structured steps if needed, but overly prescriptive guidance may limit its creativity

Now let's move on to our next technique: N-shot prompting with extended thinking.

## 2. N-shot Prompting with Extended Thinking

N-shot prompting is a powerful technique where you provide Claude with examples (or "shots") of how to approach a particular type of problem. When combined with extended thinking, this approach can be particularly effective in guiding Claude's reasoning process while still allowing it the freedom to think deeply.

### How N-shot Prompting Works with Extended Thinking

When using N-shot prompting with extended thinking, you can include examples of reasoning patterns using tags like `<thinking>` or `<scratchpad>` in your prompt. Claude will:

1. Observe the pattern of thinking in your examples
2. Generalize that pattern to its own extended thinking process
3. Apply a similar reasoning approach to the new problem

This approach helps guide the *style* of reasoning without constraining the *content* of the reasoning.

### Implementing N-shot Prompting

There are two main ways to implement N-shot prompting with extended thinking:

1. **Implicit N-shot**: Provide complete examples with both reasoning and answers
2. **Explicit N-shot**: Include explicit `<thinking>` or similar tags to highlight the reasoning process

Let's create a helper function to implement N-shot prompting with Claude:

In [ ]:
def create_nshot_prompt(examples, new_problem, thinking_tag="<thinking>"):
    """
    Create an N-shot prompt with reasoning examples
    
    Args:
        examples (list): List of dictionaries with 'problem', 'reasoning', and 'answer' keys
        new_problem (str): The new problem to solve
        thinking_tag (str): Tag to use for marking thinking sections
        
    Returns:
        str: Formatted N-shot prompt
    """
    prompt = "I'm going to show you some examples of how to solve problems, then I'll ask you to solve a new problem.\n\n"
    
    # Add the examples
    for i, example in enumerate(examples, 1):
        prompt += f"Example {i}:\n"
        prompt += f"Problem: {example['problem']}\n\n"
        prompt += f"{thinking_tag}\n{example['reasoning']}\n{thinking_tag}\n\n"
        prompt += f"Answer: {example['answer']}\n\n"
    
    # Add the new problem
    prompt += f"Now solve this problem:\n{new_problem}"
    
    return prompt

In [ ]:
# Create some examples with thinking patterns for N-shot prompting
examples = [
    {
        "problem": "If a train travels at 60 miles per hour, how far will it travel in 2.5 hours?",
        "reasoning": """
To solve this problem, I need to find the distance traveled given the speed and time.

The formula for distance is: distance = speed × time

Given:
- Speed = 60 miles per hour
- Time = 2.5 hours

Calculation:
distance = 60 miles/hour × 2.5 hours
distance = 150 miles
""",
        "answer": "The train will travel 150 miles in 2.5 hours."
    },
    {
        "problem": "A store is offering a 20% discount on an item that originally costs $85. What is the sale price?",
        "reasoning": """
To find the sale price, I need to calculate the discount amount and subtract it from the original price.

Given:
- Original price = $85
- Discount = 20%

Step 1: Calculate the discount amount.
Discount amount = Original price × Discount percentage
Discount amount = $85 × 20%
Discount amount = $85 × 0.20
Discount amount = $17

Step 2: Calculate the sale price.
Sale price = Original price - Discount amount
Sale price = $85 - $17
Sale price = $68
""",
        "answer": "The sale price is $68."
    }
]

# Create a new problem to solve using the N-shot pattern
new_math_problem = """
A car rental company charges $45 per day plus $0.25 per mile driven. 
If someone rents a car for 3 days and drives 420 miles, what is the total cost of the rental?
"""

# Create an N-shot prompt
nshot_prompt = create_nshot_prompt(examples, new_math_problem)

# Preview the prompt structure (first 500 characters)
print("N-shot prompt structure (excerpt):")
print("-" * 80)
print(nshot_prompt[:500] + "...")
print("-" * 80)

In [ ]:
# Test the N-shot prompt with extended thinking
nshot_response = claude_utils.invoke_claude(
    bedrock_runtime,
    nshot_prompt,
    CLAUDE_37_SONNET_MODEL_ID,
    enable_reasoning=True,
    reasoning_budget=4096,
    max_tokens=1000
)

# Display the result
claude_utils.display_claude_response(nshot_response)

### The Power of N-shot Prompting with Extended Thinking

As you can see from the example above, the N-shot approach guided Claude's extended thinking process to follow a similar pattern to the examples provided. This technique is particularly valuable when:

1. **You want a specific reasoning structure**: The examples guide Claude to use a particular approach
2. **You're solving domain-specific problems**: The examples can demonstrate domain conventions or terminology
3. **You want to focus the reasoning**: Examples can help Claude focus on relevant aspects of the problem

The beauty of combining N-shot prompting with extended thinking is that it provides guidance on *how to think* without constraining *what to think*. Claude still has the freedom to tackle the unique aspects of each new problem.

Let's try a more complex example where the reasoning pattern is more sophisticated:

In [ ]:
# Create examples for a more complex reasoning pattern - business analysis
business_examples = [
    {
        "problem": "A SaaS company is experiencing a 5% monthly churn rate. What strategies could help reduce this churn?",
        "reasoning": """
To address the churn problem, I'll analyze potential causes and solutions through multiple lenses:

1) Customer Perspective Analysis:
- Customers may be churning due to: poor onboarding, missing features, bad support, or finding better alternatives
- Need to consider both new and longstanding customers who might churn for different reasons

2) Product-Market Fit Analysis:
- 5% monthly churn (60% annual) suggests potential product-market fit issues
- Need to identify if this affects all customer segments or just specific ones

3) Competitive Analysis:
- What are competitors offering that this company isn't?
- Where might the product be falling behind market standards?

4) Value Proposition Review:
- Is the product delivering on its promised value?
- Is pricing aligned with the perceived value?

Based on this analysis, my solution framework will address:
- Immediate tactical fixes to address acute churn issues
- Strategic improvements to enhance product-market fit
- Process changes to better monitor and prevent future churn
""",
        "answer": """
To reduce the 5% monthly churn rate, I recommend a three-pronged approach:

1. Immediate tactical improvements:
   - Implement exit surveys to identify specific pain points
   - Create a "save team" focused on retention for at-risk customers
   - Enhance onboarding and customer success touchpoints

2. Strategic product enhancements:
   - Develop a feature roadmap based on customer feedback
   - Consider pricing tier adjustments to better match value perception
   - Introduce loyalty incentives for long-term commitments

3. Process improvements:
   - Establish early warning systems to identify at-risk customers
   - Implement regular customer health scoring
   - Create cross-functional retention teams

These strategies should aim to bring churn down to industry benchmark levels (2-3% monthly) within two quarters.
"""
    }
]

# New business problem to solve
new_business_problem = """
An e-commerce company has a 68% shopping cart abandonment rate, which is hurting their conversion metrics.
What strategies would you recommend to reduce cart abandonment and increase completed purchases?
"""

# Create an N-shot prompt with the business example
business_nshot_prompt = create_nshot_prompt(business_examples, new_business_problem)

# Test the business N-shot prompt
business_response = claude_utils.invoke_claude(
    bedrock_runtime,
    business_nshot_prompt,
    CLAUDE_37_SONNET_MODEL_ID,
    enable_reasoning=True,
    reasoning_budget=8192,  # Using a larger budget for this complex problem
    max_tokens=1500
)

# Display the result
claude_utils.display_claude_response(business_response)

## N-shot Pattern Variations

The examples above demonstrate the basic N-shot pattern, but there are several variations you can use depending on your needs:

1. **Zero-shot with format guidance**: Instead of examples, provide a format guideline using tags
2. **Multi-step N-shot**: Examples that show complex multi-stage reasoning processes
3. **Comparative N-shot**: Examples that show how to compare multiple approaches
4. **Incremental N-shot**: Examples that build on each other with increasing complexity

### Best Practices for N-shot Prompting with Extended Thinking

- **Start simple**: Begin with minimal examples and add more only if needed
- **Use realistic examples**: Make your examples relevant to the actual problems
- **Match complexity**: Your examples should be similar in complexity to your target problem
- **Use consistent formatting**: Maintain the same structure across all examples
- **Let Claude generalize**: Don't be overly prescriptive; let Claude adapt the pattern

Let's move on to our next technique: controlling reasoning depth and scope.

## 3. Techniques to Control Reasoning Depth and Scope

While extended thinking gives Claude more space to reason, sometimes you need to focus that reasoning in specific directions or control its depth. There are several effective techniques to guide Claude's extended thinking process:

1. **Focus questions**: Directing attention to specific aspects of a problem
2. **Constraint specification**: Explicitly stating boundaries and limitations
3. **Depth indicators**: Specifying how thoroughly to explore certain areas
4. **Framework application**: Suggesting specific analytical frameworks

These techniques help you tailor Claude's extended thinking to your specific needs without being overly prescriptive about the exact steps to follow.

In [ ]:
def test_reasoning_control(problem, control_technique, technique_name, reasoning_budget=4096):
    """
    Test a specific reasoning control technique
    
    Args:
        problem (str): The base problem to solve
        control_technique (str): The control technique to apply
        technique_name (str): Name of the technique for display
        reasoning_budget (int): The reasoning budget to allocate
        
    Returns:
        dict: The full API response
    """
    full_prompt = f"{control_technique}\n\n{problem}"
    
    print(f"\n=== Testing {technique_name} ===\n")
    print(f"Technique: {control_technique}\n")
    print(f"Problem: {problem}\n")
    
    response = claude_utils.invoke_claude(
        bedrock_runtime,
        full_prompt,
        CLAUDE_37_SONNET_MODEL_ID,
        enable_reasoning=True,
        reasoning_budget=reasoning_budget,
        max_tokens=1200
    )
    
    # Display key metrics
    elapsed_time = response.get('_elapsed_time', 0)
    total_tokens = response.get('usage', {}).get('totalTokens', 0)
    output_tokens = response.get('usage', {}).get('outputTokens', 0)
    
    print(f"Time: {elapsed_time:.2f}s, Tokens: {total_tokens} (Output: {output_tokens})")
    
    return response

In [ ]:
# Base problem that we'll apply different control techniques to
base_problem = """
A retail company is planning to open a new store and needs to decide between three potential locations. 
The company wants to maximize profit while considering factors like foot traffic, rent costs, 
competition, and local demographics.

Location A: Downtown - High foot traffic, high rent ($12,000/month), high competition, affluent demographics
Location B: Shopping mall - Medium foot traffic, medium rent ($8,000/month), medium competition, mixed demographics
Location C: Suburban area - Low foot traffic, low rent ($5,000/month), low competition, family demographics
"""

# Test Focus Questions technique
focus_technique = """
Think deeply about this business decision problem. In your analysis, focus particularly on:

1. How does each location's foot traffic translate to potential sales volume?
2. What is the break-even point for each location given the different rent costs?
3. How might the competitive landscape affect market share at each location?
4. Which demographic groups are most likely to become high-value customers?
"""

focus_response = test_reasoning_control(
    base_problem, 
    focus_technique, 
    "Focus Questions Technique"
)

# Display a preview of the response
result_text = claude_utils.extract_response_content(focus_response)
preview = result_text[:300] + "..." if len(result_text) > 300 else result_text
print("\nResponse preview:")
print("-" * 80)
print(preview)
print("-" * 80)

In [ ]:
# Test Constraint Specification technique
constraint_technique = """
Please analyze this business location decision thoroughly. 

As you reason through this problem, consider the following constraints:
- The company needs to achieve profitability within 12 months
- Staffing costs will be roughly the same at all locations
- The company's brand appeals most strongly to millennials and young professionals
- The store requires at least 2,000 sq ft of floor space
- The company can invest no more than $150,000 in initial setup costs
"""

constraint_response = test_reasoning_control(
    base_problem, 
    constraint_technique, 
    "Constraint Specification Technique"
)

# Display a preview of the response
result_text = claude_utils.extract_response_content(constraint_response)
preview = result_text[:300] + "..." if len(result_text) > 300 else result_text
print("\nResponse preview:")
print("-" * 80)
print(preview)
print("-" * 80)

In [ ]:
# Test Framework Application technique
framework_technique = """
Please analyze this business location decision by applying a SWOT analysis 
(Strengths, Weaknesses, Opportunities, Threats) to each potential location.

In your thinking, first create a comprehensive SWOT analysis for each location, 
then use that analysis to make an informed recommendation.
"""

framework_response = test_reasoning_control(
    base_problem, 
    framework_technique, 
    "Framework Application Technique"
)

# Display the full response for this technique
print("\nFull response from Framework Application Technique:")
claude_utils.display_claude_response(framework_response)

### Key Insights on Controlling Reasoning

As shown in the examples above, there are several effective ways to guide Claude's reasoning without being overly prescriptive about the exact steps:

1. **Focus Questions** direct Claude's attention to specific aspects of the problem that you consider most important. This is useful when you want deep analysis of particular dimensions.

2. **Constraint Specification** provides boundaries and real-world limitations that Claude should consider. This grounds the analysis in practical realities and prevents impractical solutions.

3. **Framework Application** suggests specific analytical tools or frameworks to structure the reasoning. This is particularly useful for domain-specific problems where established analytical approaches exist.

Each of these techniques guides *what* Claude reasons about while still allowing it the freedom to determine *how* to approach the reasoning. This balance is key to getting the most out of extended thinking.

## Check-in Point: Controlling Reasoning

Before moving on, confirm that you understand:

1. You can guide Claude's reasoning while still benefiting from extended thinking
2. Different control techniques serve different purposes
3. The goal is to focus the reasoning on relevant aspects without over-constraining it
4. These techniques can be combined for more nuanced control

Now let's explore how to create reusable prompt templates that incorporate these techniques.

## 4. Creating Reusable Prompt Templates

Once you've developed effective prompting techniques for extended thinking, you can create reusable templates for different use cases. These templates:

1. Save time by avoiding the need to craft custom prompts for each task
2. Ensure consistency in how Claude approaches similar problems
3. Allow for gradual refinement as you learn what works best
4. Make it easier for teams to adopt best practices

Let's build a simple template system for creating prompts that leverage extended thinking effectively:

In [ ]:
class PromptTemplate:
    """
    A reusable template for creating prompts that work well with extended thinking
    """
    def __init__(self, template, name="Unnamed Template", description=""):
        """
        Initialize a prompt template
        
        Args:
            template (str): The template string with placeholders for variables
            name (str): A descriptive name for this template
            description (str): A description of what this template is designed for
        """
        self.template = template
        self.name = name
        self.description = description
    
    def format(self, **kwargs):
        """
        Format the template by filling in the placeholders
        
        Args:
            **kwargs: The variables to fill in the template
            
        Returns:
            str: The formatted prompt
        """
        try:
            return self.template.format(**kwargs)
        except KeyError as e:
            missing_key = str(e).strip("'")
            raise ValueError(f"Missing required variable: {missing_key}")
    
    def __str__(self):
        return f"PromptTemplate('{self.name}')"
    
    def get_info(self):
        """Get information about this template"""
        return {
            "name": self.name,
            "description": self.description,
            "required_variables": self._extract_variables()
        }
    
    def _extract_variables(self):
        """Extract variable names from the template string"""
        import re
        # Find all instances of {variable_name} in the template
        matches = re.findall(r'\{([^{}]+)\}', self.template)
        return list(set(matches))  # Return unique variable names

In [ ]:
# Create a collection of reusable templates for different use cases

# Business analysis template
business_analysis_template = PromptTemplate(
    name="Business Problem Analysis",
    description="Template for analyzing business problems with multiple dimensions",
    template="""
Please think deeply and methodically about the following business problem:

{problem_statement}

In your analysis, consider:
- The key stakeholders and their interests
- Short-term and long-term implications
- Financial and non-financial impacts
- Potential risks and mitigation strategies
- Implementation challenges

{additional_context}

Develop a comprehensive recommendation based on your analysis.
"""
)

# Technical problem solving template
technical_problem_template = PromptTemplate(
    name="Technical Problem Solver",
    description="Template for solving technical or mathematical problems",
    template="""
I need a thorough solution to the following technical problem:

{problem_statement}

Please think about this problem comprehensively. Consider:
- Different approaches to solving it
- Edge cases and potential issues
- Efficiency and optimization opportunities
- Implementation considerations

{constraints}

Explain your reasoning thoroughly before providing the final solution.
"""
)

# Creative brainstorming template
creative_template = PromptTemplate(
    name="Creative Brainstorming",
    description="Template for creative idea generation and development",
    template="""
I need creative ideas for the following scenario:

{scenario}

Please think expansively and consider unconventional approaches. In your thinking:
- Generate a wide range of possibilities
- Consider cross-domain inspiration
- Think about how to combine different elements in novel ways
- Evaluate the most promising ideas in more depth

{constraints}

Develop 3-5 of your best ideas into more detailed concepts.
"""
)

# List our available templates
templates = [business_analysis_template, technical_problem_template, creative_template]
for template in templates:
    info = template.get_info()
    print(f"Template: {info['name']}")
    print(f"Description: {info['description']}")
    print(f"Required variables: {', '.join(info['required_variables'])}")
    print()

In [ ]:
# Test our templates with real problems

# Business analysis example
business_prompt = business_analysis_template.format(
    problem_statement="""
Our e-commerce company is experiencing a high rate of cart abandonment (72%).
We need to understand why customers are abandoning their carts and how we can reduce this rate.
""",
    additional_context="""
Our target demographic is primarily 25-45 year olds.
We sell premium home goods with an average order value of $120.
Our main competitors have abandonment rates of around 60%.
"""
)

# Technical problem example
technical_prompt = technical_problem_template.format(
    problem_statement="""
Design a system to efficiently store and retrieve time-series data from IoT devices.
Each device sends temperature readings every 5 minutes, and we have 10,000 devices.
We need to query average temperatures across devices for any given time period.
""",
    constraints="""
The system should be able to handle at least 5 years of historical data.
Query response time should be under 1 second for common queries.
Storage costs should be minimized.
"""
)

# Creative problem example
creative_prompt = creative_template.format(
    scenario="""
A mid-sized city wants to increase community engagement with its public parks
while also addressing issues of sustainability and technological integration.
""",
    constraints="""
Proposals should be implementable within a budget of $500,000.
Solutions should appeal to diverse age groups and backgrounds.
Priority should be given to ideas that can be implemented within 12 months.
"""
)

# Pick one example to test with Claude
print("Testing business analysis template with Claude:")
print("-" * 80)
print(business_prompt)
print("-" * 80)

business_response = claude_utils.invoke_claude(
    bedrock_runtime,
    business_prompt,
    CLAUDE_37_SONNET_MODEL_ID,
    enable_reasoning=True,
    reasoning_budget=8192,
    max_tokens=1500
)

# Display the result
claude_utils.display_claude_response(business_response)

### Template Library Benefits

As demonstrated, creating a library of prompt templates offers several advantages:

1. **Consistency**: Each template encapsulates best practices for a particular type of task
2. **Reusability**: Templates can be reused across different problems of the same type
3. **Customization**: Templates accept variables to customize the prompt for specific scenarios
4. **Team alignment**: Templates help teams adopt consistent approaches to similar problems

When building your template library, focus on:

- **Purpose-specific templates**: Create templates for specific types of tasks rather than generic ones
- **Clear variable names**: Use descriptive names for placeholders to make templates easier to use
- **Thoughtful structure**: Design templates that guide Claude's reasoning without constraining it
- **Continuous improvement**: Refine templates based on the quality of responses they generate

Let's test another template to see how it performs with extended thinking:

In [ ]:
# Let's test the technical problem template
print("Testing technical problem template with Claude:")
print("-" * 80)
print(technical_prompt)
print("-" * 80)

technical_response = claude_utils.invoke_claude(
    bedrock_runtime,
    technical_prompt,
    CLAUDE_37_SONNET_MODEL_ID,
    enable_reasoning=True,
    reasoning_budget=8192,
    max_tokens=1500
)

# Display the result
claude_utils.display_claude_response(technical_response)

### Building a Complete Template System

For production environments, you might want to expand the template system with additional features:

1. **Template validation**: Checking that templates follow best practices
2. **Performance tracking**: Monitoring which templates produce the best results
3. **Version control**: Tracking changes to templates over time
4. **Template sharing**: Making it easy for teams to share effective templates

The simple `PromptTemplate` class we created can be extended to include these features as needed for your specific use case.

Here's an example of how you might enhance the system to include performance tracking:

In [ ]:
class TrackedPromptTemplate(PromptTemplate):
    """
    An extended prompt template that tracks performance metrics
    """
    def __init__(self, template, name="Unnamed Template", description=""):
        super().__init__(template, name, description)
        self.usage_history = []
    
    def execute(self, client, model_id, variables, reasoning_budget=4096, max_tokens=1000):
        """
        Execute the template and track performance
        
        Args:
            client: The bedrock_runtime client
            model_id (str): The model ID to use
            variables (dict): Variables to fill in the template
            reasoning_budget (int): The reasoning budget to allocate
            max_tokens (int): Maximum tokens for the response
            
        Returns:
            dict: The full API response
        """
        # Format the prompt
        prompt = self.format(**variables)
        
        # Invoke Claude
        start_time = time.time()
        response = claude_utils.invoke_claude(
            client,
            prompt,
            model_id,
            enable_reasoning=True,
            reasoning_budget=reasoning_budget,
            max_tokens=max_tokens
        )
        elapsed_time = time.time() - start_time
        
        # Track performance
        metrics = {
            "timestamp": time.time(),
            "prompt_length": len(prompt),
            "variables": list(variables.keys()),
            "reasoning_budget": reasoning_budget,
            "elapsed_time": elapsed_time,
            "input_tokens": response.get('usage', {}).get('inputTokens', 0),
            "output_tokens": response.get('usage', {}).get('outputTokens', 0),
            "total_tokens": response.get('usage', {}).get('totalTokens', 0)
        }
        
        self.usage_history.append(metrics)
        
        return response
    
    def get_performance_summary(self):
        """Get a summary of performance metrics for this template"""
        if not self.usage_history:
            return "No usage history available"
        
        # Calculate averages
        avg_time = sum(m["elapsed_time"] for m in self.usage_history) / len(self.usage_history)
        avg_tokens = sum(m["total_tokens"] for m in self.usage_history) / len(self.usage_history)
        
        return {
            "name": self.name,
            "usage_count": len(self.usage_history),
            "avg_time": avg_time,
            "avg_tokens": avg_tokens,
            "last_used": max(m["timestamp"] for m in self.usage_history)
        }

# Create a tracked template
tracked_creative_template = TrackedPromptTemplate(
    name="Tracked Creative Brainstorming",
    description="Template for creative idea generation with performance tracking",
    template=creative_template.template
)

# Test the tracked template
tracked_response = tracked_creative_template.execute(
    bedrock_runtime,
    CLAUDE_37_SONNET_MODEL_ID,
    {
        "scenario": """
        A technology startup wants to create a new mobile app that encourages people 
        to reduce their carbon footprint through gamification.
        """,
        "constraints": """
        The app should be engaging enough to retain users for at least 3 months.
        The business model should be sustainable without requiring users to pay.
        The app should provide measurable environmental impact.
        """
    },
    reasoning_budget=6144
)

# Display performance metrics
print("Performance metrics:")
print(tracked_creative_template.get_performance_summary())

# Display a preview of the response
result_text = claude_utils.extract_response_content(tracked_response)
preview = result_text[:500] + "..." if len(result_text) > 500 else result_text
print("\nResponse preview:")
print("-" * 80)
print(preview)
print("-" * 80)

## Summary: Effective Prompting Techniques for Extended Thinking

In this notebook, we've explored several powerful techniques for crafting prompts that get the most out of Claude 3.7 Sonnet's extended thinking capability:

1. **General vs. Step-by-Step Instructions**
   - General instructions typically work better with extended thinking
   - Allow Claude to determine its own reasoning path while providing broad guidance
   - Find a balance that provides structure without over-constraining the thinking process

2. **N-shot Prompting with Extended Thinking**
   - Provide examples of reasoning patterns using tags like `<thinking>`
   - Guide the style of reasoning without constraining the content
   - Use realistic examples that match the complexity of your target problem

3. **Controlling Reasoning Depth and Scope**
   - Use focus questions to direct attention to specific aspects
   - Specify constraints to ground the analysis in practical realities
   - Apply frameworks to structure the reasoning process

4. **Creating Reusable Prompt Templates**
   - Develop purpose-specific templates for different types of tasks
   - Include variables to customize prompts for specific scenarios
   - Track performance to continuously improve your templates

By applying these techniques, you can craft prompts that leverage Claude's extended thinking capabilities to their fullest potential, resulting in more thorough, insightful, and valuable responses.

## Next Steps

Now that you understand how to craft effective prompts for extended thinking, you might want to explore:

1. **Tool Use Integration**: Combining extended thinking with tool calling (covered in Lesson 5)
2. **Long-Form Content Generation**: Using extended thinking to generate comprehensive content (Lesson 6)
3. **Complex Problem-Solving**: Applying extended thinking to tackle challenging problems in various domains (Lesson 7)

## Practice Exercises

To reinforce what you've learned, try these exercises:

1. Create a general instruction prompt for a complex problem in your domain
2. Develop an N-shot prompt with examples relevant to your use case
3. Create a reusable template for a common task you perform
4. Compare the results of different prompting techniques on the same problem

Remember that effective prompting is both an art and a science. Continuous experimentation and refinement will help you develop prompts that consistently produce high-quality results with Claude's extended thinking capability.